In [1]:
%cd ..

/Users/georgi/dev/dialogue_modeling


In [2]:
import spacy
import string
from read_data import read_solution_annotaions, read_wason_dump, read_3_lvl_annotation_file
from wason_message import WasonConversation, WasonMessage
import pandas as pd
from solution_tracker.simple_sol import process_raw_to_solution_tracker

In [3]:
from collections import defaultdict

In [4]:
anns = read_solution_annotaions('solution_annotations.tsv')
nlp = spacy.load("en_core_web_sm")
for a in anns:
    a.preprocess_everything(nlp)

raw_data = read_wason_dump('data/all/')

conversations_to_process = []
for ann in anns:
    raw = [r for r in raw_data if r.identifier == ann.identifier][0]
    ann.raw_db_conversation = raw.raw_db_conversation
    conversations_to_process.append(ann)

hierch_data = read_3_lvl_annotation_file('3lvl_anns.tsv')

for conv in anns:
    hierch = [d for d in hierch_data if d.identifier == conv.identifier][0]
    conv.merge_all_annotations(hierch)



sols = defaultdict(lambda x: [])
for conv in conversations_to_process:
    _, sol_tracker = process_raw_to_solution_tracker(conv)
    sols[conv.identifier] = sol_tracker


yes two cards {'U', '3'} {'0'}
we could to 3,4, u or w, 3, 4, u but I'm going with 4, u {'U', '3', '4', 'W'} {'U', '4'}
my guess is all of them except 1 {'9', 'J', '4', 'I'} {'9', 'J', '4'}
@Ox, I did the same {'0'} {'4', 'I'}
1 and last then? {'0'} {'4', 'I'}
I'm going with first and last like I started with. {'I'} {'4', 'I'}
Me too {'I'} {'4', 'I'}
I think if it was U and 2 then you wouldn't be able to see if the theory was incorrect so i reckon its U and 7 {'U', '2', '7'} {'U', '7'}
it Would be the letters? {'0'} {'A', 'C'}
Yeah {'0'} {'5', 'A'}
do we all agree that 5 needs testing or {'5', '4', 'E', 'L'} {'5'}
yeah {'5', '4', 'E', 'L'} {'5'}
Yeah {'5', '4', 'E', 'L'} {'5'}
and e? {'E'} {'5', 'E'}
Yeah {'E'} {'5', 'E'}
Okay I put 6 because I thpught we need to check if there's a vowel on the other side, and then S to make sure there's not an even number on that {'6', 'S', 'A'} {'6', 'S'}
Do we all agree on 7 and A? {'6', 'S', '7', 'A'} {'7', 'A'}
Yes {'6', 'S', '7', 'A'} {'7', 'A'}


In [5]:
import copy
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', 0)

In [6]:

allowed = {
    'vowels': {'A', 'O', 'U', 'E', 'I', 'Y'},
    'consonants': {'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'X', 'Z','W'},
    'odds': {'1', '3', '5', '7', '9'},
    'evens': {'0', '2', '4', '6', '8'}    
}


#all cards with vowels on one side have even numbers on the other
# To prove the rule, one should turn the vowel and the odd number

def is_solution_fine_grained(state):
    checked_dict = {'vowels': False, 'consonants': False, 'odds': False, 'evens': False}
  
    for item in state:
        
        for checked in checked_dict.keys():
            if item in allowed[checked]:
                checked_dict[checked] = True
    
    score = 0
    classification = 'OTHER_ERROR'
    for key, item in checked_dict.items():
        if key == 'vowels' and item:
            score += 0.25
        elif key == 'odds' and item:
            score += 0.25
        elif key == 'consonants' and not item:
            score += 0.25
        elif key == 'evens' and not item:
            score += 0.25
    
    if score == 1:
        classification = 'CORRECT'
    elif score == 0:
        classification = 'ALL_INCORRECT'
        
    if checked_dict['vowels'] and checked_dict['evens']:
        if checked_dict['odds']:
            classification = 'BIASED + ODDS'
        elif checked_dict['consonants']:
            classification = 'BIASED + CONST'
        else:
            classification = 'BIASED'
        
    if all(value for value in checked_dict.values()):
        classification = 'ALL_CHECKED'
    
    if all(value is False for value in checked_dict.values()):
        classification = 'NONE_CHECKED'
        
    return score, classification

In [7]:
def calculate_team_performance(latest_solutions):
    score = 0
    for u, solution in latest_solutions.items():
        local_score, _ = is_solution_fine_grained(solution)
        score += local_score
    
    return round(score / len(latest_solutions), 3)

In [18]:
for k_sol, sol in sols.items():
#     print(k_sol)
    
    wason_conv = [a for a in anns if a.identifier == k_sol][0]
    latest_sol = {}
    latest_score = 0
    participation_tracker = {}
    participation_normalised = {}
    messages = 0
    for item in sol:
        if item['type'] == 'INITIAL':
            latest_sol[item['user']] = " ".join(item['value'])
        else: 
            if item['user'] not in latest_sol:
                latest_sol[item['user']] = 'N/A'

    for user in latest_sol.keys():
        participation_normalised[user + '_participation'] = 0
        participation_tracker[user] = 0

    df = pd.DataFrame(columns=['origin', 'content', *latest_sol.keys(), 'team_performance', 'performance_change', *participation_normalised.keys()])        
    team_performance = calculate_team_performance(latest_sol)

    display_dict = copy.copy(latest_sol)
    display_dict['team_performance'] = team_performance
    display_dict['performance_change'] = 0
    latest_score = team_performance
    
    df = df.append({'origin': 'initial', 'content': 'Solo Submissions', **display_dict},
                   ignore_index=True)
    
    for raw in wason_conv.raw_db_conversation:
        if raw['user_status'] != 'USR_PLAYING':
            continue
            
        if raw['message_type'] == 'WASON_SUBMIT':
            local_sol = [s for s in sol if s['id'] == raw['message_id']][0]
            norm_value = " ".join(local_sol['value'])   
        elif raw['message_type'] == 'CHAT_MESSAGE':
            local_sol = [s for s in sol if s['id'] == raw['message_id']]
            if len(local_sol) == 0:
                local_sol = {'user': raw['user_name'], 'value': latest_sol[raw['user_name']]}
                norm_value = latest_sol[raw['user_name']]
            else:
                local_sol = local_sol[0]
                norm_value = " ".join(local_sol['value'])
            
            messages += 1
            participation_tracker[raw['user_name']] += 1
            for k_abs, v_abs in participation_tracker.items():
                participation_normalised[k_abs + '_participation'] = v_abs / messages
        else:
            continue
                
        if norm_value == latest_sol[local_sol['user']]:
            display_value = norm_value
        else:
            display_value = "*** {} ***".format(norm_value)

        latest_sol[local_sol['user']] = norm_value
        team_performance = calculate_team_performance(latest_sol)
        display_dict = copy.copy(latest_sol)
        display_dict[local_sol['user']] = display_value
        display_dict['team_performance'] = team_performance
        display_dict['performance_change'] = team_performance - latest_score
        latest_score = team_performance
        df = df.append({'origin': local_sol['user'],
                        'content': 'WASON_SUMBIT' if raw['message_type'] == 'WASON_SUBMIT' 
                                else raw['content'],
                        **display_dict, **participation_normalised},
           ignore_index=True)
        

    h = HTML(df.to_html())
    display(h)
    print('-----------------------------------')

,origin,content,Alpaca,Leopard,Chipmunk,team_performance,performance_change,Alpaca_participation,Leopard_participation,Chipmunk_participation
0,initial,Solo Submissions,U 3,U 4,3,0.750,0,NaN,NaN,NaN
1,Leopard,hi,U 3,U 4,3,0.750,0,0.000000,1.000000,0.000000
2,Chipmunk,Hello,U 3,U 4,3,0.750,0,0.000000,0.500000,0.500000
3,Alpaca,hiya,U 3,U 4,3,0.750,0,0.333333,0.333333,0.333333
4,Leopard,is it 4 and u?,U 3,U 4,3,0.750,0,0.250000,0.500000,0.250000
5,Chipmunk,So what do you guys think?,U 3,U 4,3,0.750,0,0.200000,0.400000,0.400000
6,Chipmunk,I think we only need to do 2 right?,U 3,U 4,3,0.750,0,0.166667,0.333333,0.500000
7,Alpaca,I believe the answer is 3 and u.,U 3,U 4,3,0.750,0,0.285714,0.285714,0.428571
8,Leopard,I don't have a 2,U 3,U 4,3,0.750,0,0.250000,0.375000,0.375000
9,Chipmunk,No I meant like two cards,U 3,U 4,3,0.750,0,0.222222,0.333333,0.444444


-----------------------------------


,origin,content,Beaver,Duck,Ox,team_performance,performance_change,Beaver_participation,Duck_participation,Ox_participation
0,initial,Solo Submissions,4 I,9 J 4,4 I,0.417,0,NaN,NaN,NaN
1,Ox,Hello,4 I,9 J 4,4 I,0.417,0,0.000000,0.000000,1.000000
2,Duck,Hi,4 I,9 J 4,4 I,0.417,0,0.000000,0.500000,0.500000
3,Beaver,Hello,4 I,9 J 4,4 I,0.417,0,0.333333,0.333333,0.333333
4,Beaver,What do you guys think is the best solution?,4 I,9 J 4,4 I,0.417,0,0.500000,0.250000,0.250000
5,Duck,my guess is all of them except 1,4 I,*** 9 J 4 I ***,4 I,0.500,0.083,0.400000,0.400000,0.200000
6,Ox,"Did you pick I and 4 as your cards, I did. They would be the only ones you would have to check to verify the rule, right?",4 I,9 J 4 I,4 I,0.500,0,0.333333,0.333333,0.333333
7,Beaver,Why do you think that @Duck?,4 I,9 J 4 I,4 I,0.500,0,0.428571,0.285714,0.285714
8,Duck,All cards with vowels on one side have an even number on the other,4 I,9 J 4 I,4 I,0.500,0,0.375000,0.375000,0.250000
9,Duck,it's already given,4 I,9 J 4 I,4 I,0.500,0,0.333333,0.444444,0.222222


-----------------------------------


,origin,content,Chipmunk,Duck,Zebra,Beaver,team_performance,performance_change,Chipmunk_participation,Duck_participation,Zebra_participation,Beaver_participation
0,initial,Solo Submissions,U 2,B 2 7 U,U 7,B 2 7 U,0.625,0,NaN,NaN,NaN,NaN
1,Chipmunk,WASON_SUMBIT,U 2,B 2 7 U,U 7,B 2 7 U,0.625,0,0.000000,0.000000,0.000000,0.000000
2,Duck,WASON_SUMBIT,U 2,*** U ***,U 7,B 2 7 U,0.688,0.063,0.000000,0.000000,0.000000,0.000000
3,Zebra,What answers did everyone put?,U 2,U,U 7,B 2 7 U,0.688,0,0.000000,0.000000,1.000000,0.000000
4,Duck,Okay so what answer did everyone put to see if we all put the same answer?,U 2,U,U 7,B 2 7 U,0.688,0,0.000000,0.500000,0.500000,0.000000
5,Zebra,I put 7 and U,U 2,U,U 7,B 2 7 U,0.688,0,0.000000,0.333333,0.666667,0.000000
6,Beaver,I put U and 2,U 2,U,U 7,*** U 2 ***,0.688,0,0.000000,0.250000,0.500000,0.250000
7,Chipmunk,I selected U and 2 as they are vowels and even number,U 2,U,U 7,U 2,0.688,0,0.200000,0.200000,0.400000,0.200000
8,Duck,WASON_SUMBIT,U 2,*** U 2 ***,U 7,U 2,0.625,-0.063,0.200000,0.200000,0.400000,0.200000
9,Zebra,I think it would be 7 to test if it would be incorrect,U 2,U 2,*** 7 ***,U 2,0.562,-0.063,0.166667,0.166667,0.500000,0.166667


...

-----------------------------------


,origin,content,Llama,Cat,Kiwi,Bee,team_performance,performance_change,Llama_participation,Cat_participation,Kiwi_participation,Bee_participation
0,initial,Solo Submissions,E,5 E,N/A,N/A,0.688,0,NaN,NaN,NaN,NaN
1,Cat,i think its 5 and e,E,5 E,N/A,N/A,0.688,0,0.0,1.000000,0.000000,0.000000
2,Kiwi,I also think it's 5 and E,E,5 E,*** 5 E ***,N/A,0.812,0.124,0.0,0.500000,0.500000,0.000000
3,Cat,because the E can prove the rule and then the 5 could disprove it,E,*** E 5 ***,5 E,N/A,0.812,0,0.0,0.666667,0.333333,0.000000
4,Bee,How many cards can you pick?,E,E 5,5 E,N/A,0.812,0,0.0,0.500000,0.250000,0.250000
5,Kiwi,You can pick 2,E,E 5,5 E,N/A,0.812,0,0.0,0.400000,0.400000,0.200000
6,Kiwi,Oh,E,E 5,5 E,N/A,0.812,0,0.0,0.333333,0.500000,0.166667
7,Cat,i think as many as you like,E,E 5,5 E,N/A,0.812,0,0.0,0.428571,0.428571,0.142857
8,Cat,but it says must,E,E 5,5 E,N/A,0.812,0,0.0,0.500000,0.375000,0.125000
9,Kiwi,Actually you could turn them all!,E,E 5,*** G 5 8 E ***,N/A,0.688,-0.124,0.0,0.444444,0.444444,0.111111


-----------------------------------


,origin,content,Lion,Ox,Bat,team_performance,performance_change,Lion_participation,Ox_participation,Bat_participation
0,initial,Solo Submissions,U 4,9 U,U 4,0.667,0,NaN,NaN,NaN
1,Lion,WASON_SUMBIT,U 4,9 U,U 4,0.667,0,0.000000,0.000000,0.000000
2,Bat,Hello,U 4,9 U,U 4,0.667,0,0.000000,0.000000,1.000000
3,Lion,hihi,U 4,9 U,U 4,0.667,0,0.500000,0.000000,0.500000
4,Ox,Hello :),U 4,9 U,U 4,0.667,0,0.333333,0.333333,0.333333
5,Bat,What do you guys think is the answer?,U 4,9 U,U 4,0.667,0,0.250000,0.250000,0.500000
6,Bat,"Also just to check are the cards you can see 9,L,4,U?",U 4,9 U,U 4,0.667,0,0.200000,0.200000,0.600000
7,Ox,I chose 9 and U.,U 4,9 U,U 4,0.667,0,0.166667,0.333333,0.500000
8,Lion,Yes i have 9L4U too,*** 9 U ***,9 U,U 4,0.833,0.166,0.285714,0.285714,0.428571
9,Ox,"Yes, I have those.",9 U,9 U,U 4,0.833,0,0.250000,0.375000,0.375000


-----------------------------------


In [9]:
h

,origin,content,Lion,Ox,Bat,team_performance,performance_change
0,initial,Solo Submissions,U 4,9 U,U 4,0.667,0
1,Lion,WASON_SUMBIT,U 4,9 U,U 4,0.667,0
2,Bat,Hello,U 4,9 U,U 4,0.667,0
3,Lion,hihi,U 4,9 U,U 4,0.667,0
4,Ox,Hello :),U 4,9 U,U 4,0.667,0
5,Bat,What do you guys think is the answer?,U 4,9 U,U 4,0.667,0
6,Bat,"Also just to check are the cards you can see 9,L,4,U?",U 4,9 U,U 4,0.667,0
7,Ox,I chose 9 and U.,U 4,9 U,U 4,0.667,0
8,Lion,Yes i have 9L4U too,*** 9 U ***,9 U,U 4,0.833,0.166
9,Ox,"Yes, I have those.",9 U,9 U,U 4,0.833,0


In [10]:
df

,origin,content,Lion,Ox,Bat,team_performance,performance_change
0,initial,Solo Submissions,U 4,9 U,U 4,0.667,0
1,Lion,WASON_SUMBIT,U 4,9 U,U 4,0.667,0
2,Bat,Hello,U 4,9 U,U 4,0.667,0
3,Lion,hihi,U 4,9 U,U 4,0.667,0
4,Ox,Hello :),U 4,9 U,U 4,0.667,0
5,Bat,What do you guys think is the answer?,U 4,9 U,U 4,0.667,0
6,Bat,"Also just to check are the cards you can see 9,L,4,U?",U 4,9 U,U 4,0.667,0
7,Ox,I chose 9 and U.,U 4,9 U,U 4,0.667,0
8,Lion,Yes i have 9L4U too,*** 9 U ***,9 U,U 4,0.833,0.166
9,Ox,"Yes, I have those.",9 U,9 U,U 4,0.833,0


In [11]:
latest_sol

{'Lion': '9 U', 'Ox': '9 U', 'Bat': '9 U'}